In [1]:
from numcodecs import Blosc

In [2]:
from faim_ipa.hcs.cellvoyager.source import CVSourceHDF5

In [3]:
h5_src = CVSourceHDF5(
    '/tungstenfs/scratch/gscicomp_share/gmicro/test-data-with-logs/jetraw_blosc2.h5',
    'C07-data'
)

In [4]:
from faim_ipa.hcs.cellvoyager.acquisition import ZAdjustedStackAcquisition, TileAlignmentOptions

plate = ZAdjustedStackAcquisition(
    source=CVSourceFS(
        acquisition_dir='/tungstenfs/scratch/gscicomp_share/gmicro/test-data-with-logs/jetraw/C07-data/',
        trace_log_files=[
            '/tungstenfs/scratch/gscicomp_share/gmicro/test-data-with-logs/jetraw/LOG/TRACE2024-02-19.log',
            '/tungstenfs/scratch/gscicomp_share/gmicro/test-data-with-logs/jetraw/LOG/TRACE2024-02-20.log',
            '/tungstenfs/scratch/gscicomp_share/gmicro/test-data-with-logs/jetraw/LOG/TRACE2024-02-21.log',
        ],
    ),
    alignment=TileAlignmentOptions.STAGE_POSITION,
    n_planes_in_stacked_tile=16
)

In [5]:
plate = ZAdjustedStackAcquisition(
    source=h5_src,
    alignment=TileAlignmentOptions.STAGE_POSITION,
    n_planes_in_stacked_tile=16
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.18s/it]


In [6]:
tile = plate.get_well_acquisitions()[0].get_tiles()[0]

In [7]:
img = tile.load_data()

In [8]:
img.shape

(16, 2000, 2000)

In [9]:
c07 = plate.get_well_acquisitions(["C07"])[0]

In [10]:
tiles = c07.get_tiles()

In [11]:
from distributed import Client, wait
import dask.array as da
from faim_ipa.stitching import DaskTileStitcher, stitching_utils
import zarr
from ome_zarr.io import parse_url
from ome_zarr.writer import write_multiscales_metadata

In [12]:
client = Client(
        n_workers=4,
        threads_per_worker=2,
        memory_limit="4GB",
        local_directory="./dask_tmp/"
    )

2024-09-05 19:01:45,072 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:33413' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('store-map-147e925b43fc1b23ade5342de4dfc469', 3, 7, 3, 1), ('block-info-assemble_chunk-f1fdd15bee34f4d9ff13ee34937d3b77', 0, 3, 14, 1, 3), ('store-map-147e925b43fc1b23ade5342de4dfc469', 3, 9, 2, 1), ('store-map-147e925b43fc1b23ade5342de4dfc469', 3, 6, 1, 0), ('store-map-147e925b43fc1b23ade5342de4dfc469', 3, 4, 1, 1), ('store-map-147e925b43fc1b23ade5342de4dfc469', 3, 14, 2, 2), ('store-map-147e925b43fc1b23ade5342de4dfc469', 3, 3, 0, 0), ('store-map-147e925b43fc1b23ade5342de4dfc469', 3, 7, 1, 0), ('store-map-147e925b43fc1b23ade5342de4dfc469', 3, 3, 0, 3), ('store-map-147e925b43fc1b23ade5342de4dfc469', 3, 5, 0, 1), ('store-map-147e925b43fc1b23ade5342de4dfc469', 3, 7, 1, 3), ('store-map-147e925b43fc1b23ade5342de4dfc469', 3, 3, 2, 3), ('store-map-147e925b43fc1b23ade5342de4dfc469', 3, 8, 0, 1), ('store-m

In [13]:
stitcher = DaskTileStitcher(
        tiles=tiles,
        chunk_shape=(16, 2000, 2000),
        output_shape=c07.get_shape(),
        dtype=c07.get_dtype(),
    )

In [14]:
image_da = stitcher.get_stitched_dask_array(
    warp_func=stitching_utils.translate_tiles_2d,
    fuse_func=stitching_utils.fuse_linear, # here a fuse-jetraw function would be used.
).squeeze()

In [15]:
storage_options =dict(
    # Here the jetraw compressor would be used?
    compressor=Blosc(cname="zstd", clevel=3, shuffle=Blosc.SHUFFLE),
    chunks=(16, 2000, 2000),
    write_empty_chunks=False,
)

In [16]:
group = zarr.group(parse_url("./C07.zarr", mode="w").store, overwrite=True)

In [17]:
write_multiscales_metadata(
    group=group,
    datasets=[
        {
            "path": "0",
            "coordinateTransformations": c07.get_coordinate_transformations(
                max_layer=0,
                yx_binning=1,
                ndim=4,
            )[0]
        }
    ],
    axes=["c", "z", "y", "x"],
)

In [ ]:
wait(
    client.persist(
        da.to_zarr(
            arr=image_da,
            url=group.store,
            component="0",
            storage_options=storage_options,
            compressor=storage_options["compressor"],
            dimension_separator="/",
        )
    )
)

2024-09-05 19:01:45,062 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('assemble_chunk-getitem-getitem-d959ac4a46cc7ac6670bb2b2d506a00a', 3, 14, 2, 0))" coro=<Worker.execute() done, defined at /home/tibuch/Gitrepos/faim-ipa/examples/hdf5-to-zarr/.pixi/envs/default/lib/python3.12/site-packages/distributed/worker_state_machine.py:3609>> ended with CancelledError
2024-09-05 19:01:45,063 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('assemble_chunk-getitem-getitem-d959ac4a46cc7ac6670bb2b2d506a00a', 3, 14, 1, 3))" coro=<Worker.execute() done, defined at /home/tibuch/Gitrepos/faim-ipa/examples/hdf5-to-zarr/.pixi/envs/default/lib/python3.12/site-packages/distributed/worker_state_machine.py:3609>> ended with CancelledError
2024-09-05 19:01:45,064 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('store-map-147e925b43fc1b23ade5342de4dfc469'

In [ ]:
client.shutdown()